# Modeling (GARCH, VAR/Granger/IRFs, Wavelets)
This notebook fits GARCH(1,1) per asset, a VAR model for BTC↔S&P spillovers, and a discrete wavelet transform (DWT) for scale analysis.

**Outputs saved:**
- `figures/*_garch_sigma.png`
- `tables/garch_params.csv`
- `figures/var_impulse_responses.png`, `tables/var_summary.txt`
- `figures/wavelet_energy.png`, `tables/wavelet_energy.csv`


In [ ]:
import os, numpy as np, pandas as pd, matplotlib.pyplot as plt

BASE = "/content" if os.path.exists("/content") else "."
DATA = f"{BASE}/data"
FIG  = f"{BASE}/figures"; os.makedirs(FIG, exist_ok=True)
TAB  = f"{BASE}/tables";  os.makedirs(TAB, exist_ok=True)

plt.rcParams["figure.figsize"] = (10, 5)
print("BASE:", BASE)


> If running in a fresh Colab, install deps using `requirements.txt`.

In [ ]:
!pip -q install -r requirements.txt


## Load returns
Prefer `data/returns_daily.csv` (from Stooq). If missing, build returns from `data/merged_returns.csv`.

In [ ]:
import pandas as pd, numpy as np, os

rets_path = f"{DATA}/returns_daily.csv"
merged_path = f"{DATA}/merged_returns.csv"

if os.path.exists(rets_path):
    rets = pd.read_csv(rets_path, parse_dates=["Date"]).set_index("Date").sort_index()
    print("Loaded daily returns from", rets_path, rets.shape)
else:
    if not os.path.exists(merged_path):
        raise FileNotFoundError("Neither data/returns_daily.csv nor data/merged_returns.csv found.")
    df = pd.read_csv(merged_path, parse_dates=["Date"]).set_index("Date").sort_index()
    BTC_CLOSE = "BTC_Close"
    ETH_CLOSE = "ETH_Close" if "ETH_Close" in df.columns else None
    SPX_RET   = "SPX_Close_Return"
    DJI_CLOSE = "DJI_Close" if "DJI_Close" in df.columns else None
    cols = {}
    if "BTC_Close_Return" in df.columns:
        cols["BTC-USD"] = df["BTC_Close_Return"]
    else:
        if BTC_CLOSE not in df.columns:
            raise KeyError("BTC_Close_Return missing and BTC_Close not present to compute it.")
        cols["BTC-USD"] = np.log(df[BTC_CLOSE]).diff()*100
    if ETH_CLOSE and ("ETH_Close_Return" in df.columns):
        cols["ETH-USD"] = df["ETH_Close_Return"]
    elif ETH_CLOSE:
        cols["ETH-USD"] = np.log(df[ETH_CLOSE]).diff()*100
    if SPX_RET in df.columns:
        cols["^GSPC"] = df[SPX_RET]
    else:
        if "SPX_Close" in df.columns:
            cols["^GSPC"] = np.log(df["SPX_Close"]).diff()*100
    if "DJI_Close_Return" in df.columns:
        cols["^DJI"] = df["DJI_Close_Return"]
    elif DJI_CLOSE:
        cols["^DJI"] = np.log(df[DJI_CLOSE]).diff()*100

    rets = pd.DataFrame(cols).dropna(how="all").dropna()
    print("Built returns from merged file:", merged_path, rets.shape)

display(rets.tail())


## GARCH(1,1) per asset
Fits a standard GARCH(1,1) with Gaussian errors; exports parameter table and conditional volatility plots.

In [ ]:
from arch import arch_model
from statsmodels.stats.diagnostic import acorr_ljungbox

targets = [c for c in rets.columns if c in ["BTC-USD","ETH-USD","^GSPC","^DJI"]]
if not targets:
    targets = list(rets.columns)[:4]

rows = []
for col in targets:
    series = rets[col].dropna()
    am = arch_model(series, vol="GARCH", p=1, q=1, mean="Constant", dist="normal")
    res = am.fit(disp="off")
    params = res.params
    alpha = params.get("alpha[1]", np.nan)
    beta  = params.get("beta[1]" , np.nan)
    rows.append({
        "asset": col,
        "mu": params.get("mu", np.nan),
        "omega": params.get("omega", np.nan),
        "alpha": alpha,
        "beta": beta,
        "alpha_plus_beta": (alpha + beta) if np.isfinite(alpha) and np.isfinite(beta) else np.nan,
        "llf": res.loglikelihood,
        "ljungbox_p(10)": acorr_ljungbox(res.std_resid.dropna(), lags=[10], return_df=True)["lb_pvalue"].values[0]
    })
    res.conditional_volatility.plot()
    plt.title(f"GARCH(1,1) Conditional Volatility — {col}")
    plt.xlabel("Date"); plt.ylabel("Volatility (σ_t)")
    out = f"{FIG}/{col.replace('^','')}_garch_sigma.png"
    plt.tight_layout(); plt.savefig(out, dpi=150); plt.close()
    print("Saved:", out)

garch_tbl = pd.DataFrame(rows)
garch_tbl.to_csv(f"{TAB}/garch_params.csv", index=False)
display(garch_tbl)
print("Saved table:", f"{TAB}/garch_params.csv")


## VAR(p) with AIC lag selection, Granger causality, IRFs
We focus on BTC and S&P 500 to quantify short-horizon spillovers.

In [ ]:
from statsmodels.tsa.api import VAR

need = [c for c in ["BTC-USD","^GSPC"] if c in rets.columns]
if len(need) < 2:
    need = list(rets.columns[:2])
    print("Using first two columns for VAR:", need)

Y = rets[need].dropna()
model = VAR(Y)
res = model.fit(ic="aic", trend="c")
lag = res.k_ar
print("AIC-selected lag:", lag)

with open(f"{TAB}/var_summary.txt","w") as f:
    f.write(str(res.summary()))
    try:
        f.write("

Granger ({} <- {}):
".format(need[0], need[1]))
        f.write(str(res.test_causality(need[0], [need[1]], kind="f").summary()))
    except Exception as e:
        f.write(f"

Granger test 1 failed: {e}")
    try:
        f.write("

Granger ({} <- {}):
".format(need[1], need[0]))
        f.write(str(res.test_causality(need[1], [need[0]], kind="f").summary()))
    except Exception as e:
        f.write(f"

Granger test 2 failed: {e}")
print("Saved:", f"{TAB}/var_summary.txt")

irf = res.irf(10)
fig = irf.plot(orth=False)
out = f"{FIG}/var_impulse_responses.png"
fig.savefig(out, dpi=150); plt.close(fig)
print("Saved:", out)


## Wavelet decomposition (db4, level=3)
Decompose BTC returns and plot energy distribution across components.

In [ ]:
import pywt

series_name = "BTC-USD" if "BTC-USD" in rets.columns else rets.columns[0]
series = rets[series_name].dropna().values
wave, level = "db4", 3

coeffs = pywt.wavedec(series, wavelet=wave, level=level)
energies = [float(np.sum(c**2)) for c in coeffs]
labels = [f"A{level}"] + [f"D{j}" for j in range(level, 0, -1)]

plt.bar(labels, energies)
plt.title(f"Wavelet Energy — {series_name} ({wave}, level={level})")
plt.xlabel("Component"); plt.ylabel("Energy")
out = f"{FIG}/wavelet_energy.png"
plt.tight_layout(); plt.savefig(out, dpi=150); plt.close()
print("Saved:", out)

pd.DataFrame({"component": labels, "energy": energies}).to_csv(f"{TAB}/wavelet_energy.csv", index=False)
print("Saved:", f"{TAB}/wavelet_energy.csv")
